In [1]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 2.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.51.1
    Uninstalling grpcio-1.51.1:
      Successfully uninstalled grpcio-1.51.1


In [2]:
!pip install keras-image-helper

In [4]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

## Setup grpc connection

In [5]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [12]:
## Preprocess Image & Setup prediction service

In [6]:
from keras_image_helper import create_preprocessor

In [7]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [8]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [10]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [11]:
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [14]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [15]:
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.8798645734786987
    float_val: -4.756311893463135
    float_val: -2.3595333099365234
    float_val: -1.0892642736434937
    float_val: 9.90378475189209
    float_val: -2.826181173324585
    float_val: -3.648310422897339
    float_val: 3.2411563396453857
    float_val: -2.612095832824707
    float_val: -4.852035999298096
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [17]:
preds = pb_response.outputs['dense_7'].float_val

In [18]:
classes=[
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds))

{'dress': -1.8798645734786987,
 'hat': -4.756311893463135,
 'longsleeve': -2.3595333099365234,
 'outwear': -1.0892642736434937,
 'pants': 9.90378475189209,
 'shirt': -2.826181173324585,
 'shoes': -3.648310422897339,
 'shorts': 3.2411563396453857,
 'skirt': -2.612095832824707,
 't-shirt': -4.852035999298096}